<a href="https://colab.research.google.com/github/KooEric/tensorflow/blob/main/musicVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 진행사항(From github)
1. 자체 데이터로 모델을 학습시키려면 먼저 Magenta 환경을 설정
2. 다음으로, 데이터세트 만들기의 지침에 따라 MIDI 파일 모음을 NoteSequences의 TFRecord로 변환  
3. configs.py 에서 미리 정의된 구성 중 하나를 선택 하거나 직접 정의
4. 마지막으로 training 스크립트 를 실행 
> 해당 과정(전처리 -> 학습 -> 생성)의 프로세스

VAE는 AE와 형태 면에서 유사하며, 학습 데이터(training data)를 병목(Bottleneck) 구조를 통해서 압축하여 저차원의 잠재 코드(latent code, z)를 만듭니다. 잠재 공간(latent space)을 통해서 모델은 입력 데이터 간의 유사성과 차이를 학습합니다.

VAE의 인코더는 입력 데이터를 평균과 표준 편차 벡터로 인코딩 한 후, 두 벡터에 대응하는 분포에서 샘플링을 수행합니다. 그리고 쿨백-라이블러 발산(KL divergence)을 손실함수로 사용하여, 해당 분포가 표준 정규 분포에 가까워지도록 학습하게 됩니다. 결국, 표준 정규 분포에 근접한 분포에서 샘플링된 VAE의 잠재 공간 분포는 원점을 기준으로 데이터가 대칭적이고, 분포가 고른 형태를 보이게 됩니다.

Variational AutoEncoder(변분추론 인코더)의 경우 입력데이터 X를 잘 설명하는 특징(feature)을 추출하여 Latent vector(잠재백터) Z에 담은후에, 이 Latent vector(잠재백터) Z를 통하여, X와 유사하나 완전 새로운 데이터를 생성하는것을 목표로합니다.

In [1]:
# Dependency issue 해결
!pip install magenta==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 24.0 MB/s 
     |████████████████████████████████| 5.6 MB 56.4 MB/s 
     |████████████████████████████████| 90 kB 5.7 MB/s 
     |████████████████████████████████| 352 kB 56.4 MB/s 
     |████████████████████████████████| 209 kB 64.5 MB/s 
     |████████████████████████████████| 1.4 MB 64.0 MB/s 
     |████████████████████████████████| 204 kB 76.6 MB/s 
     |████████████████████████████████| 254 kB 51.6 MB/s 
     |████████████████████████████████| 3.6 MB 69.7 MB/s 
     |████████████████████████████████| 10.9 MB 67.7 MB/s 
     |████████████████████████████████| 69 kB 5.4 MB/s 
     |████████████████████████████████| 2.3 MB 58.6 MB/s 
     |████████████████████████████████| 47 kB 3.0 MB/s 
     |████████████████████████████████| 270 kB 71.7 MB/s 
     |████████████████████████████████| 508 kB 65.4 MB/s 
     |██████████████████████████████

In [38]:
# import
import tensorflow as tf
import numpy as np
import pathlib
import zipfile
import os
import pandas as pd
import IPython
import collections
import note_seq 
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data
from magenta.scripts.convert_dir_to_note_sequences import convert_directory #전처리함수
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import tensorflow.compat.v1 as tf # tensorflow 1과 2의 호환성을 위해 진행 
tf.disable_v2_behavior()
import tf_slim 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#데이터 zip파일 불러오기
url = "https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip"
dir = tf.keras.utils.get_file(origin=url, 
                                   fname='/content/groove.zip', 
                                   extract=True)
data_dir = pathlib.Path(dir)

3268608/3260318 [==============================] - 0s 0us/step


In [5]:
#zip파일 압축해제
zipfile.ZipFile('/content/groove.zip').extractall()

In [6]:
#경로지정
data_root= '/content/groove'
# midi파일의 session, id, bpm info확인을 위함
csv_file = '/content/groove/info.csv' 
# tfrecord파일 경로를 지정
tfrec_root = '/content/drum.tfrecord'

In [25]:
import pandas as pd
df = pd.read_csv('/content/groove/info.csv')
df = pd.DataFrame(df)
df.head(10)

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
5,drummer1,drummer1/eval_session,drummer1/eval_session/5,funk/groove5,84,beat,4-4,drummer1/eval_session/5_funk-groove5_84_beat_4...,drummer1/eval_session/5_funk-groove5_84_beat_4...,45.687518,test
6,drummer1,drummer1/eval_session,drummer1/eval_session/6,hiphop/groove6,87,beat,4-4,drummer1/eval_session/6_hiphop-groove6_87_beat...,drummer1/eval_session/6_hiphop-groove6_87_beat...,44.119242,test
7,drummer1,drummer1/eval_session,drummer1/eval_session/7,pop/groove7,138,beat,4-4,drummer1/eval_session/7_pop-groove7_138_beat_4...,drummer1/eval_session/7_pop-groove7_138_beat_4...,27.706547,test
8,drummer1,drummer1/eval_session,drummer1/eval_session/8,rock/groove8,65,beat,4-4,drummer1/eval_session/8_rock-groove8_65_beat_4...,drummer1/eval_session/8_rock-groove8_65_beat_4...,59.067313,test
9,drummer1,drummer1/eval_session,drummer1/eval_session/9,soul/groove9,105,beat,4-4,drummer1/eval_session/9_soul-groove9_105_beat_...,drummer1/eval_session/9_soul-groove9_105_beat_...,36.540504,test


midi데이터를 학습하기위해, 백터화 해야하며, 백터화하여 TFrecord로 저장할 것 입니다.
midi 포멧으로 읽어서 매번 디코딩을 진행하면 학습단계에서 데이터를 읽을때 성능저하가 발생합니다. 이를 해결하기위해 TFrecord파일을 사용할 수 있습니다.

In [8]:
convert_directory(data_root,tfrec_root,recursive=True) #전처리 함수

In [10]:
#configs.py 26-46, 500-523 부분을 가져와 선언

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)

HParams = contrib_training.HParams

def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)


CONFIG_MAP = {}

# 모델 config configs.py의 groovae config 사용

#=== 모델구조 ===
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(), #BidirectionalLstmEncoder 사용 
                   lstm_models.GrooveLstmDecoder()), #Hierarchical Decoder
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512, #데이터 배치사이즈
            max_seq_len=16 * 4,  # 4마디 길이지정
            z_size=256, #잠재백터 사이즈
            enc_rnn_size=[512], #인코더 순환 사이즈지정
            dec_rnn_size=[256, 256],#디코더 순환사이즈 지정
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,#드롭아웃
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,
        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path='/content/drum.tfrecord', #데이터 경로 설정
)

#Training
TFRecord 데이터를 입력 시퀀스로 하여 VAE 모델을 학습시킬 수 있습니다. MusicVAE의 configs.py에는 다양한 설정이 있고, Groove MIDI Dataset으로 MusicVAE 모델을 학습시키고 4마디에 해당하는 드럼 샘플을 생성하는 과정입니다.

In [49]:
# music_vae_train.py 초기 flag를 제외하고 _trial_summary, get_input_tensors, train 함수를 가져온다.
# Should not be called from within the graph to avoid redundant summaries.
def _trial_summary(hparams, examples_path, output_dir):
  #텐서보드 summary 텍스트

    examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])

    hparams_dict = hparams.values()

#하이퍼 파라미터
  # Create a markdown table from hparams.
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()


def _get_input_tensors(dataset, config):
  #데이터로부터 텐서 입력 
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,
    sequence_length) = iterator.get_next()
    input_sequence.set_shape(
    [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
    [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth])
        sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())
        
    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

#훈련 체크포인트
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):

# train
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
            num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            #옵티마이저
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')
            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op = train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)



def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.compat.v1.python_io.tf_record_iterator,
        is_training=True):
    config = config_map['groovae_4bar']
    train_dir = '/content/train'
    num_steps = 10000 #훈련 epoch
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            num_steps=num_steps)      
    
    else:
        print("EVAL")

In [ ]:
run(CONFIG_MAP)

/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.


훈련된 체크포인트를 이용해서 새로운 MIDI 데이터를 생성할 수 있습니다. 생성 방식에는 ‘Sample’ 방식과 ‘Interpolate’ 방식이 있습니다.

In [ ]:
#music_vae/music_vae_generate.py
model = TrainedModel(
    config=CONFIG_MAP['groovae_4bar'],
    batch_size=1,
    checkpoint_dir_or_path='/content/train')#checkpoint

generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], '/content/drum_4bar.mid')